# Exploratory Data Analysis

- Dataset can be found here: https://www.kaggle.com/datasets/grosvenpaul/family-income-and-expenditure
    - Dataset is from **2015-2018** the latest publicly available nationwide Family Income and Expenditure Survey (FIES) conducted every three (3) years by the Philippine Statistics Authority (PSA).


# Data Wrangling

In [ ]:
# Import the required libraries for EDA
import pandas as pd
import numpy as np

In [ ]:
# Loading the CSV file into a Pandas DataFrame
df = pd.read_csv('Family_Income_and_Expenditure.csv')

In [ ]:
# Checking the shape of the DataFrame
df.shape

In [ ]:
df.columns

- Dataset has **41544 instances** and **60 features**. The instances represented here are different Filipino Families from all over the Philippines, feature represent Total Household Income, Region, Total Food Expenditure and other factors dealing with Income and Expenditure.

In [ ]:
# Removing irrelevant features
df.drop(['Total Food Expenditure', 'Agricultural Household indicator', 'Household Head Sex',
         'Household Head Age', 'Household Head Marital Status', 'Type of Household',
         'Members with age less than 5 year old', 'Members with age 5 - 17 years old', 
         'Total number of family members employed', 'Type of Building/House',
         'Type of Roof', 'Type of Walls', 'House Age', 'Number of bedrooms', 'Tenure Status', 
         'Toilet Facilities', 'Number of Television', 'Number of CD/VCD/DVD', 
         'Number of Component/Stereo set', 'Number of Refrigerator/Freezer',
         'Number of Washing Machine', 'Number of Airconditioner', 'Number of Car, Jeep, Van', 
         'Number of Landline/wireless telephones', 'Number of Cellular phone', 
         'Number of Personal Computer', 'Number of Stove with Oven/Gas Range', 
         'Number of Motorized Banca', 'Number of Motorcycle/Tricycle'], axis=1, inplace=True)

In [ ]:
# Identify null values
df.isnull().sum()

In [ ]:
# Check duplicates
df.duplicated().any()

Inference:
1. Since the output is False, we can say that there's No Duplicate Records present in the Dataset.

# Preprocess the data
### Basic Information of Attributes

In [ ]:
# Check information
df.info()

Inference:
1. There are only **24 Numerical Attributes** in the dataset.
2. On the other hand we have **9 Categorical Attributes**.

- Display the numerical features in the dataset

In [ ]:
df.select_dtypes(np.number).head()

Inference:
1. All numerical features are appropriate for the analysis.
2. Add new column with categorical values that describes the numerical values of 'Electricity'

In [ ]:
# Identify and Create new column for 'Has Electricity'
has_electricity = pd.Series('No Electricity', index=df.index).mask(df['Electricity'] == 1, 'With Electricity')
df.insert(30, 'Has Electricity', has_electricity)

# Compute for Monthly Income and Monthly Income per Capita
monthly_income = (df['Total Household Income'] // 12)
monthly_capita = round(monthly_income / df['Total Number of Family members'], 2)
df.insert(2, 'Monthly Income per Capita', monthly_capita)

# Identifty Income Class of Families
# Based on PIDS Report (2018)
poor = df[monthly_capita <= 2191.4]
low = (monthly_capita >= 2191.4) & (monthly_capita <= 4238.8)
middle = (monthly_capita >= 4238.8) & (monthly_capita <= 26296.8)
high = (monthly_capita >= 26296.8) & (monthly_capita <= 43828)
rich = (monthly_capita >= 43828)

# Create Column 'Income Class'
income_class = pd.Series('Poor', index=df.index).mask(
    ((monthly_capita >= 2191.4) & (monthly_capita <= 4238.8)), 
        'Low Income').mask(middle, 'Middle Class').mask(high, 
        'High Income').mask(rich, 'Rich')
df.insert(3, 'Income Class', income_class)

- Display the categorical features in the dataset

In [ ]:
df.select_dtypes(include="O").head()

- Check Unique Values of Categorical Attributes

In [ ]:
cat_columns = df.select_dtypes(include="O").columns

for column in cat_columns:
    print('Unique values of ', column, set(df[column]))
    print("-"*127)

In [ ]:
# Fill null values'
df['Household Head Occupation'].fillna('Unemployed', inplace=True)
df['Household Head Class of Worker'].fillna('Unemployed', inplace=True)

- Group similar values for 'Household Head Highest Grade Completed

In [ ]:
# Grade 1-6, change to Elementary Undergraduate
df = df.replace({'Household Head Highest Grade Completed':{'Grade 1':'Elementary Undergraduate', 'Grade 2':'Elementary Undergraduate', 'Grade 3':'Elementary Undergraduate', 'Grade 4':'Elementary Undergraduate', 'Grade 5':'Elementary Undergraduate', 'Grade 6':'Elementary Undergraduate'}})
# First-Fourth Year High School, change to High School Undergraduate
df = df.replace({'Household Head Highest Grade Completed':{'First Year High School':'High School Undergraduate', 'Second Year High School':'High School Undergraduate', 'Third Year High School':'High School Undergraduate', 'Fourth Year High School':'High School Undergraduate'}})
# First-Fourth Year College, change to College Undergraduate
df = df.replace({'Household Head Highest Grade Completed':{'First Year College':'High School Undergraduate', 'Second Year College':'High School Undergraduate', 'Third Year College':'High School Undergraduate', 'Fourth Year College':'High School Undergraduate'}})
# Replace rows containing 'Programs' with 'Degree Programs or Equivalent'
df.loc[df['Household Head Highest Grade Completed'].str.contains('Programs'),'Household Head Highest Grade Completed'] = 'Degree Programs or Equivalent'
# Replace rows containing 'Post Secondary' with 'Post Secondary Undergraduate'
df.loc[df['Household Head Highest Grade Completed'].str.contains('Post Secondary'),'Household Head Highest Grade Completed'] = 'Post Secondary Undergraduate'

- Group similar values for 'Household Head Class of Worker'

In [ ]:
# Replace 'Worked for private establishment' with 'Private Employment'
df = df.replace({'Household Head Class of Worker':{'Worked for private establishment':'Private Employment'}})
# Replace 'Worked for private household' with 'Private Employment'
df = df.replace({'Household Head Class of Worker':{'Worked for private household':'Private Employment'}})
# Replace 'Employer in own family-operated farm or business' with 'Employer in Family Business'
df = df.replace({'Household Head Class of Worker':{'Self-employed wihout any employee':'Self Employed'}})
# Replace 'Employer in own family-operated farm or business' with 'Employer in Family Business'
df = df.replace({'Household Head Class of Worker':{'Employer in own family-operated farm or business':'Self Employed'}})
# Replace 'Worked for government/government corporation' with 'Government Employment'
df = df.replace({'Household Head Class of Worker':{'Worked for government/government corporation':'Government Employment'}})
# Replace 'Worked without pay in own family-operated farm or business' with 'Employee in Family Business(w/o pay)'
df = df.replace({'Household Head Class of Worker':{'Worked without pay in own family-operated farm or business':'Employee in Family Business'}})
# Replace 'Worked with pay in own family-operated farm or business' with 'Employee in Family Business(w/ pay)'
df = df.replace({'Household Head Class of Worker':{'Worked with pay in own family-operated farm or business':'Employee in Family Business'}})

- Group similar values for 'Household Head Occupation

In [ ]:
# Replace rows containing 'farmers' with 'Farmers'
df.loc[df['Household Head Occupation'].str.contains('farmers'),'Household Head Occupation'] = 'Farmers'
# Replace rows containing 'laborers' with 'Laborers'
df.loc[df['Household Head Occupation'].str.contains('laborers'),'Household Head Occupation'] = 'Laborers'
# Replace rows containing 'engineers' with 'Engineers'
df.loc[df['Household Head Occupation'].str.contains('engineer'),'Household Head Occupation'] = 'Engineers'
df.loc[df['Household Head Occupation'].str.contains('Engineer'),'Household Head Occupation'] = 'Engineers'
# Replace rows containing 'drivers' with 'Drivers'
df.loc[df['Household Head Occupation'].str.contains('drivers'),'Household Head Occupation'] = 'Drivers'
# Replace rows containing 'managers' with 'Managers'
df.loc[df['Household Head Occupation'].str.contains('managers'),'Household Head Occupation'] = 'Managers'
# Replace rows containing 'fish' with 'Fishermen'
df.loc[df['Household Head Occupation'].str.contains('fish'),'Household Head Occupation'] = 'Fishermen'
# Replace rows containing 'professionals' with 'Professionals'
df.loc[df['Household Head Occupation'].str.contains('professionals'),'Household Head Occupation'] = 'Professionals'
# Replace rows containing 'school' with 'Schoolworkers'
df.loc[df['Household Head Occupation'].str.contains('school'),'Household Head Occupation'] = 'Schoolworkers'
df.loc[df['Household Head Occupation'].str.contains('School'),'Household Head Occupation'] = 'Schoolworkers'
# Replace rows containing 'clerks' with 'Clerks'
df.loc[df['Household Head Occupation'].str.contains('clerks'),'Household Head Occupation'] = 'Clerks'
# Replace rows containing 'government' with 'Public Official'
df.loc[df['Household Head Occupation'].str.contains('government'),'Household Head Occupation'] = 'Public Official'
df.loc[df['Household Head Occupation'].str.contains('Government'),'Household Head Occupation'] = 'Public Official'
# Replace rows containing 'n. e. c.' with 'n.e.c. trained Professionals'
df.loc[df['Household Head Occupation'].str.contains('n. e. c.'),'Household Head Occupation'] = 'n.e.c. trained professionals'
# Replace rows containing 'n. e. c.' with 'n.e.c. trained Professionals'
df.loc[df['Household Head Occupation'].str.contains('n. e. c.'),'Household Head Occupation'] = 'n.e.c. trained professionals'
# Replace rows containing 'operators' with 'Operators'
df.loc[df['Household Head Occupation'].str.contains('operators'),'Household Head Occupation'] = 'Operators'
# Replace rows containing 'workers' with 'Operators'
df.loc[df['Household Head Occupation'].str.contains('workers'),'Household Head Occupation'] = 'Workers'
# Replace rows containing 'cleaners' with 'Cleaners'
df.loc[df['Household Head Occupation'].str.contains('cleaners'),'Household Head Occupation'] = 'Cleaners'
# Replace rows containing 'Police' with 'Police'
df.loc[df['Household Head Occupation'].str.contains('Police'),'Household Head Occupation'] = 'Police'
# Replace rows containing 'supervisors' with 'Supervisors'
df.loc[df['Household Head Occupation'].str.contains('supervisors'),'Household Head Occupation'] = 'Supervisors'
# Group medical related occupations into 'Medical Professional'
df = df.replace({'Household Head Occupation':{'Medical technologists':'Medical Professional', 'Physiotherapists':'Medical Professional',
                                              'Optometrists and opticians':'Medical Professional', 'Veterinarians':'Medical Professional', 
                                              'Dentists':'Medical Professional', 'Medical doctors':'Medical Professional', 
                                              'Pharmacists':'Medical Professional', 'Medical assistants':'Medical Professional'}})


In [ ]:
# Correct the values of 'Region'
df = df.replace({'Region':{'Caraga':'Region XIII', 'VI - Western Visayas':'Region VI', 
                           'V - Bicol Region':'Region V', ' ARMM':'BARMM', 'III - Central Luzon':'Region III',
                           'II - Cagayan Valley':'Region II', 'IVA - CALABARZON':'Region IV-A', 
                           'VII - Central Visayas':'Region VII','X - Northern Mindanao':'Region X', 'XI - Davao Region':'Region XI',
                           'VIII - Eastern Visayas':'Region VIII', 'I - Ilocos Region':'Region I', 'IVB - MIMAROPA':'Region IV-B', 
                           'XII - SOCCSKSARGEN':'Region XII','IX - Zasmboanga Peninsula':'Region IX'}})

In [ ]:
# Sort the DataFrame
df = df.sort_values(by=['Total Household Income'], ascending = True)

In [ ]:
df.to_csv("Capstone_DW.csv")

In [ ]:
df_mean = round(df.groupby('Region').mean(numeric_only = True).reset_index(), 2)

In [ ]:
df_mean.to_csv('Capstone_Average.csv')